In [11]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("logregconsult").getOrCreate()
data = spark.read.csv("customer_churn.csv", header=True, inferSchema=True)
data.head()

Row(Names='Cameron Williams', Age=42.0, Total_Purchase=11066.8, Account_Manager=0, Years=7.22, Num_Sites=8.0, Onboard_date=datetime.datetime(2013, 8, 30, 7, 0, 40), Location='10265 Elizabeth Mission Barkerburgh, AK 89518', Company='Harvey LLC', Churn=1)

In [14]:
print(data.columns)
print("\n")
print(data.printSchema())

['Names', 'Age', 'Total_Purchase', 'Account_Manager', 'Years', 'Num_Sites', 'Onboard_date', 'Location', 'Company', 'Churn']


root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)

None


In [20]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=['Age', 'Total_Purchase', 'Account_Manager', 'Years', 'Num_Sites'], outputCol="features")
output = assembler.transform(data)

In [21]:
final_data = output.select("features", "churn")
final_data.show()

+--------------------+-----+
|            features|churn|
+--------------------+-----+
|[42.0,11066.8,0.0...|    1|
|[41.0,11916.22,0....|    1|
|[38.0,12884.75,0....|    1|
|[42.0,8010.76,0.0...|    1|
|[37.0,9191.58,0.0...|    1|
|[48.0,10356.02,0....|    1|
|[44.0,11331.58,1....|    1|
|[32.0,9885.12,1.0...|    1|
|[43.0,14062.6,1.0...|    1|
|[40.0,8066.94,1.0...|    1|
|[30.0,11575.37,1....|    1|
|[45.0,8771.02,1.0...|    1|
|[45.0,8988.67,1.0...|    1|
|[40.0,8283.32,1.0...|    1|
|[41.0,6569.87,1.0...|    1|
|[38.0,10494.82,1....|    1|
|[45.0,8213.41,1.0...|    1|
|[43.0,11226.88,0....|    1|
|[53.0,5515.09,0.0...|    1|
|[46.0,8046.4,1.0,...|    1|
+--------------------+-----+
only showing top 20 rows



In [23]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [26]:
from pyspark.ml.classification import LogisticRegression

lr_churn = LogisticRegression(labelCol="churn")
fitted_churn_model = lr_churn.fit(train_data)

In [30]:
training_sum = fitted_churn_model.summary
training_sum.predictions.describe().show()

+-------+------------------+------------------+
|summary|             churn|        prediction|
+-------+------------------+------------------+
|  count|               636|               636|
|   mean|0.1761006289308176|0.1289308176100629|
| stddev|0.3812055670449483|0.3353871260798749|
|    min|               0.0|               0.0|
|    max|               1.0|               1.0|
+-------+------------------+------------------+



In [32]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

pred_and_labels = fitted_churn_model.evaluate(test_data)
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[26.0,8787.39,1.0...|    1|[0.79250804171589...|[0.68836960191755...|       0.0|
|[29.0,5900.78,1.0...|    0|[3.86289197182104...|[0.97942506153524...|       0.0|
|[29.0,8688.17,1.0...|    1|[2.62028983848991...|[0.93215603844092...|       0.0|
|[29.0,9378.24,0.0...|    0|[4.45668131152946...|[0.98853223630869...|       0.0|
|[29.0,10203.18,1....|    0|[3.59684044470729...|[0.97332108435472...|       0.0|
|[29.0,12711.15,0....|    0|[4.97934922218297...|[0.99316845354760...|       0.0|
|[30.0,6744.87,0.0...|    0|[3.25394720317667...|[0.96281469109618...|       0.0|
|[30.0,13473.35,0....|    0|[2.75539305210366...|[0.94021720752452...|       0.0|
|[31.0,8688.21,0.0...|    0|[6.23373627452709...|[0.99804173511255...|       0.0|
|[31.0,9574.89,0

In [34]:
churn_eval = BinaryClassificationEvaluator(labelCol="churn", rawPredictionCol="prediction")

In [36]:
auc = churn_eval.evaluate(pred_and_labels.predictions)

In [38]:
auc

0.7784117373078714

In [39]:
###PREDICT ON NEW DATA###

In [43]:
final_lr_model = lr_churn.fit(final_data)

In [49]:
new_customers = spark.read.csv("new_customers.csv", header=True, inferSchema=True)

In [50]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [53]:
test_new_customers = assembler.transform(new_costumers)
final_results = final_lr_model.transform(test_new_customers)

In [56]:
final_results.select("Company", "prediction").show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

